In [1]:
import subprocess
import statistics
import re
from math import sqrt

def benchmark_classes(classes, num_iters, ref_means, ref_errs):
    percent_means = []
    percent_err = []

    for i, class_ in enumerate(classes):
        percent_mean, percent_err = benchmark_class(class_, num_iters, ref_means[i], ref_errs[i])
        percent_means.append(percent_mean)
        percent_err.append(percent_err)

    return percent_means, percent_err


def benchmark_class(class_, num_iters, ref_mean, ref_err):
    runtime_mean, runtime_err = execute_class(class_, num_iters)
    percent_means, percent_err = compare_to_ref(runtime_mean, runtime_err, ref_mean, ref_err)

    return percent_means, percent_err


def execute_classes(classes, num_iters):
    runtime_means = []
    runtime_errs = []

    for i, class_ in enumerate(classes):
        runtime_mean, runtime_err = execute_class(class_, num_iters)
        runtime_means.append(runtime_mean)
        runtime_errs.append(runtime_err)

    return runtime_means, runtime_errs


def execute_class(class_, num_iters):
    cmd = f'./bin/ft.{class_}'

    runtimes = []

    print(f'Class: {class_}, Iterations: {num_iters}')

    for i in range(num_iters):
        res = subprocess.check_output(cmd.split(' ')).decode('utf-8')

        match = re.search(r'Verification\s+=\s+(\w+)', res)
        assert match, 'No match found!'
        assert match.group(1) == 'SUCCESSFUL', f'Verification = {match.group(1)}'

        match = re.search(r' Time in seconds\s+=\s+(\d+\.\d+)', res)
        assert match, 'No match found!'
        runtime = float(match.group(1))
        runtimes.append(runtime)

        runtime_mean, runtime_err = calc_stats(runtimes)
        print(f'  [{(i+1):03d}] Runtime: {runtime_mean:>8.6f} ± {runtime_err:.6f} [sec] ({runtime:>8.6f})')

    print()

    return runtime_mean, runtime_err


def compare_to_ref(runtime_mean, runtime_err, ref_mean, ref_err):
    diff_mean = ref_mean - runtime_mean
    diff_err = sqrt(ref_err**2 + runtime_err**2)

    percent_mean = diff_mean / ref_mean * 100.0
    percent_err = sqrt((diff_err/diff_mean)**2 + (ref_err/ref_mean)**2) * abs(percent_mean) if diff_mean != 0.0 else 0.0

    print(f'Runtime: {runtime_mean:.6f} ± {runtime_err:.6f} [sec]')
    print(f'Ref    : {ref_mean:.6f} ± {ref_err:.6f} [sec]')
    print(f'Diff   : {diff_mean:.6f} ± {diff_err:.6f} [sec]')
    print(f'Speedup: {percent_mean:>7.2f}% ± {percent_err:>7.2f}%')

    return percent_mean, percent_err


def calc_stats(runtimes):
    if len(runtimes) == 0:
        raise ValueError("runtimes array is empty")
    elif len(runtimes) == 1:
        runtime_mean = runtimes[0]
        runtime_err = 0.0
    else:
        runtime_mean = statistics.mean(runtimes)
        runtime_stdev = statistics.stdev(runtimes)

        # Filter out outliers
        runtimes_filtered = [runtime for runtime in runtimes if abs(runtime - runtime_mean) < 2 * runtime_stdev]
        if len(runtimes_filtered) < len(runtimes):
            runtime_mean = statistics.mean(runtimes_filtered)
            runtime_stdev = statistics.stdev(runtimes_filtered)

        runtime_err = 2 * runtime_stdev

    return runtime_mean, runtime_err

In [5]:
!make cleanall
!make FT CLASS=S
!make FT CLASS=W
!make FT CLASS=A
!make FT CLASS=B
!make FT CLASS=C
!make FT CLASS=D

rm -f core 
rm -f *~ */core */*~ */*.o */npbparams.hpp */*.obj */*.exe
rm -f sys/setparams sys/makesuite sys/setparams.hpp
rm -r bin/*
   =      NAS PARALLEL BENCHMARKS 4.1        =
   =      OpenMP Versions                    =
   =      C++                                =

cd FT; make CLASS=S
make[1]: Entering directory '/home/u216110/multi-core-processing-project/ours/FT'
make[2]: Entering directory '/home/u216110/multi-core-processing-project/ours/sys'
g++ -g -fopenmp -o setparams setparams.cpp
make[2]: Leaving directory '/home/u216110/multi-core-processing-project/ours/sys'
../sys/setparams ft S
make[1]: Warning: File 'npbparams.hpp' has modification time 0.00071 s in the future
g++ -std=c++14 -g -mavx512vl  -c -I../common -I/opt/intel/oneapi/vtune/2024.1/sdk/include -O3 -fopenmp -mcmodel=medium ft.cpp
cd ../common; g++ -std=c++14 -g -mavx512vl  -c -I../common -I/opt/intel/oneapi/vtune/2024.1/sdk/include -O3 -fopenmp -mcmodel=medium c_randdp.cpp
cd ../common; g++ -std=c++14 -g -m

In [ ]:
CLASSES = ['C', 'D']
# CLASSES = ['S', 'W', 'A', 'B', 'C', 'D']
num_iters=50

runtime_means, runtime_errs = execute_classes(CLASSES, num_iters)

print(f'runtime_means: {runtime_means}')
print(f'runtime_errs : {runtime_errs}')

In [16]:
class_ = 'C'
num_iters = 100

runtime_mean, runtime_err = execute_class(class_, num_iters)

print(f'runtime_mean: {runtime_mean:.6}')
print(f'runtime_err : {runtime_err:.6}')

# ours: 3.304523 ± 0.630665

Class: C, Iterations: 100


  [001] Runtime: 12.340323 ± 0.000000 [sec] (12.340323)
  [002] Runtime: 12.148646 ± 0.542143 [sec] (11.956970)
  [003] Runtime: 12.074489 ± 0.461467 [sec] (11.926174)
  [004] Runtime: 13.071470 ± 4.005685 [sec] (16.062414)


KeyboardInterrupt: 

In [10]:
# Iterations:        1000           1000           200            100            50              2
ref_mean_by_class = {'S': 0.000000, 'W': 0.000000, 'A': 0.000000, 'B': 2.913119, 'C': 0.000000, 'D': 0.000000}
ref_err_by_class  = {'S': 0.000000, 'W': 0.000000, 'A': 0.000000, 'B': 0.629813, 'C': 0.000000, 'D': 0.000000}

class_ = 'B'
num_iters = 50

ref_mean = ref_mean_by_class[class_]
ref_err = ref_err_by_class[class_]

_ = benchmark_class(class_, num_iters, ref_mean, ref_err)

Class: B, Iterations: 50


  [001] Runtime: 3.298517 ± 0.000000 [sec] (3.298517)
  [002] Runtime: 3.089596 ± 0.590916 [sec] (2.880676)
  [003] Runtime: 3.125022 ± 0.435489 [sec] (3.195872)
  [004] Runtime: 3.032199 ± 0.514090 [sec] (2.753733)
  [005] Runtime: 3.110229 ± 0.565677 [sec] (3.422349)


KeyboardInterrupt: 